# Getting Started with Ollama 🚀

Welcome to your first experience with Ollama! This notebook will guide you through the first steps to use Ollama, showing how to make simple requests with both Python and curl.

## What you'll learn:
- How to verify that Ollama is running
- Make your first request with Python
- Make your first request with curl
- Understand the basic response structure

## Prerequisites:
- Ollama installed and running (`ollama serve`)
- Model `gemma3:1b` downloaded (`ollama pull gemma3:1b`)
- Python `requests` library installed

## Step 1: Import Libraries and Setup

First, let's import the necessary Python libraries and configure the basic parameters.

In [1]:
# Before starting the code, please create a virtual environment and install requests or "requirements.txt"

# pip install requests

import requests
import json
import subprocess
import time

# Ollama configuration
OLLAMA_URL = "http://localhost:11434"
MODEL_NAME = "gemma3:1b"
SIMPLE_QUESTION = "What is artificial intelligence?"

print("✅ Libraries imported successfully!")
print(f"🎯 Model used: {MODEL_NAME}")
print(f"❓ Test question: {SIMPLE_QUESTION}")

✅ Libraries imported successfully!
🎯 Model used: gemma3:1b
❓ Test question: What is artificial intelligence?


## Step 2: Check if Ollama is Running

Before making requests, let's verify that the Ollama server is running and our model is available.

In [2]:
def check_ollama_status():
    """Check if the Ollama server is running and if the model is available."""
    try:
        # Check if the server is running
        response = requests.get(f"{OLLAMA_URL}/api/tags", timeout=5)
        
        if response.status_code == 200:
            models = response.json().get('models', [])
            model_names = [model['name'] for model in models]
            
            print("✅ Ollama server is running!")
            print(f"📋 Available models: {len(models)}")
            
            # Show the first 5 models
            for model in models[:5]:
                print(f"   - {model['name']} ({model.get('size', 'unknown size')})")
            
            if len(models) > 5:
                print(f"   ... and {len(models) - 5} more models")
            
            # Check if our model is available
            if MODEL_NAME in model_names:
                print(f"✅ Model {MODEL_NAME} is available!")
                return True
            else:
                print(f"⚠️  Model {MODEL_NAME} is not available.")
                print(f"💡 Download it with: ollama pull {MODEL_NAME}")
                return False
        else:
            print(f"❌ Ollama server responded with code {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ Unable to connect to Ollama server: {e}")
        print("💡 Make sure Ollama is installed and running: `ollama serve`")
        return False

# Test the connection
ollama_ready = check_ollama_status()

✅ Ollama server is running!
📋 Available models: 6
   - qwen3:4b (2497293918)
   - qwen3:8b (5225388164)
   - huihui_ai/dolphin3-abliterated:8b (4920760656)
   - dolphin3:8b (4920757726)
   - gemma3:12b (8149190253)
   ... and 1 more models
✅ Model gemma3:1b is available!


## Step 3: First Request with Python

Now let's make our first request to the model using Python and the `requests` library.

In [3]:
def first_python_request(question, model=MODEL_NAME):
    """Make a simple request to the Ollama model using Python."""
    
    # Prepare the request data
    payload = {
        "model": model,
        "prompt": question,
        "stream": False  # We don't want streaming for simplicity
    }
    
    print(f"🔄 Sending request to model {model}...")
    print(f"❓ Question: {question}")
    print("\n" + "-"*50)
    
    try:
        # Record start time
        start_time = time.time()
        
        # Send the POST request
        response = requests.post(f"{OLLAMA_URL}/api/generate", json=payload, timeout=30)
        
        # Calculate elapsed time
        end_time = time.time()
        total_time = end_time - start_time
        
        if response.status_code == 200:
            result = response.json()
            
            print("✅ Request completed successfully!")
            print(f"⏱️  Time taken: {total_time:.2f} seconds")
            print(f"📊 Tokens generated: {result.get('eval_count', 'N/A')}")
            print("\n" + "="*50)
            print("🤖 MODEL RESPONSE:")
            print("="*50)
            print(result['response'])
            print("="*50)
            
            return result
        else:
            print(f"❌ HTTP Error {response.status_code}: {response.text}")
            return None
            
    except requests.exceptions.Timeout:
        print("❌ Timeout: The request took too long")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

# Execute the first Python request if Ollama is ready
if ollama_ready:
    python_result = first_python_request(SIMPLE_QUESTION)
else:
    print("⚠️  Skipping Python request because Ollama is not ready")

🔄 Sending request to model gemma3:1b...
❓ Question: What is artificial intelligence?

--------------------------------------------------
✅ Request completed successfully!
⏱️  Time taken: 11.52 seconds
📊 Tokens generated: 941

🤖 MODEL RESPONSE:
Okay, let's break down what artificial intelligence (AI) is. It’s a really broad and evolving field, but here’s a breakdown of the key concepts:

**1. At its Core: Making Machines "Think" Like Humans**

Artificial Intelligence is fundamentally about creating machines – computers, software, or robots – that can perform tasks that typically require human intelligence. This isn't just about making them *faster* or *more efficient*; it's about enabling them to *reason*, *learn*, *problem-solve*, and *understand* information in a way that mimics human cognitive abilities.

**2. Types of AI - Different Approaches**

There’s not just *one* type of AI. Here are some major categories:

* **Narrow or Weak AI:** This is the AI we see most of today. It's des

## Step 4: First Request with curl

Now let's make the same request using the `curl` command from the terminal. This is useful for testing the API directly or integrating Ollama into bash scripts.

In [4]:
def first_curl_request(question, model=MODEL_NAME):
    """Make a simple request to the Ollama model using curl."""
    
    # Prepare the curl command
    curl_command = [
        'curl',
        '-X', 'POST',
        f'{OLLAMA_URL}/api/generate',
        '-H', 'Content-Type: application/json',
        '-d', json.dumps({
            "model": model,
            "prompt": question,
            "stream": False
        })
    ]
    
    print(f"🔄 Executing curl request to model {model}...")
    print(f"❓ Question: {question}")
    print("\n📋 Equivalent curl command:")
    print(f"```bash")
    print(f"curl -X POST {OLLAMA_URL}/api/generate \\")
    print(f"  -H 'Content-Type: application/json' \\")
    print(f"  -d '{{\"model\": \"{model}\", \"prompt\": \"{question}\", \"stream\": false}}'")
    print(f"```")
    print("\n" + "-"*50)
    
    try:
        # Record start time
        start_time = time.time()
        
        # Execute the curl command
        result = subprocess.run(curl_command, capture_output=True, text=True, timeout=30)
        
        # Calculate elapsed time
        end_time = time.time()
        total_time = end_time - start_time
        
        if result.returncode == 0:
            try:
                json_response = json.loads(result.stdout)
                
                print("✅ Curl request completed successfully!")
                print(f"⏱️  Time taken: {total_time:.2f} seconds")
                print(f"📊 Tokens generated: {json_response.get('eval_count', 'N/A')}")
                print("\n" + "="*50)
                print("🤖 MODEL RESPONSE (via curl):")
                print("="*50)
                print(json_response['response'])
                print("="*50)
                
                return json_response
            except json.JSONDecodeError:
                print(f"❌ Error parsing JSON response:")
                print(result.stdout)
                return None
        else:
            print(f"❌ Curl error (code {result.returncode}):")
            print(result.stderr)
            return None
            
    except subprocess.TimeoutExpired:
        print("❌ Timeout: The curl command took too long")
        return None
    except FileNotFoundError:
        print("❌ Curl command not found. Make sure curl is installed.")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

# Execute the first curl request if Ollama is ready
if ollama_ready:
    curl_result = first_curl_request(SIMPLE_QUESTION)
else:
    print("⚠️  Skipping curl request because Ollama is not ready")

🔄 Executing curl request to model gemma3:1b...
❓ Question: What is artificial intelligence?

📋 Equivalent curl command:
```bash
curl -X POST http://localhost:11434/api/generate \
  -H 'Content-Type: application/json' \
  -d '{"model": "gemma3:1b", "prompt": "What is artificial intelligence?", "stream": false}'
```

--------------------------------------------------
✅ Curl request completed successfully!
⏱️  Time taken: 10.41 seconds
📊 Tokens generated: 897

🤖 MODEL RESPONSE (via curl):
Okay, let's break down what artificial intelligence (AI) is. It's a really broad and evolving field, but here's a breakdown of the key concepts:

**1. At its Core: Mimicking Human Intelligence**

Artificial intelligence, at its most fundamental, is about creating computer systems that can perform tasks that typically require human intelligence. This doesn’t necessarily mean replicating *exactly* how a human brain works, but rather mimicking *certain cognitive skills*.

**2. Key Types of AI**

There are s

## Step 5: Compare Responses

Let's compare the responses obtained with Python and curl to verify they are equivalent.

In [5]:
def compare_responses():
    """Compare the responses obtained with Python and curl."""
    
    if not ollama_ready:
        print("⚠️  Ollama is not ready, unable to compare responses")
        return
    
    print("🔍 RESPONSE COMPARISON")
    print("="*60)
    
    # Check if we have both responses
    python_ok = 'python_result' in globals() and python_result is not None
    curl_ok = 'curl_result' in globals() and curl_result is not None
    
    if python_ok and curl_ok:
        print("📊 Comparison statistics:")
        print(f"   - Python tokens: {python_result.get('eval_count', 'N/A')}")
        print(f"   - Curl tokens: {curl_result.get('eval_count', 'N/A')}")
        
        # Compare response lengths
        len_python = len(python_result['response'])
        len_curl = len(curl_result['response'])
        print(f"   - Python response length: {len_python} characters")
        print(f"   - Curl response length: {len_curl} characters")
        
        # Check if responses are identical
        if python_result['response'] == curl_result['response']:
            print("\n✅ The responses are identical!")
        else:
            print("\n🔄 The responses are different (normal for generative models)")
        
        print("\n💡 Note: It's normal for responses to be different between")
        print("   successive calls, even with the same parameters, due to the")
        print("   probabilistic nature of language models.")
        
    elif python_ok:
        print("✅ Only the Python request succeeded")
    elif curl_ok:
        print("✅ Only the curl request succeeded")
    else:
        print("❌ No requests succeeded")

# Execute the comparison
compare_responses()

🔍 RESPONSE COMPARISON
📊 Comparison statistics:
   - Python tokens: 941
   - Curl tokens: 897
   - Python response length: 4173 characters
   - Curl response length: 3943 characters

🔄 The responses are different (normal for generative models)

💡 Note: It's normal for responses to be different between
   successive calls, even with the same parameters, due to the
   probabilistic nature of language models.


## Step 6: Understanding the Response Structure

Let's analyze the complete structure of the JSON response to understand what information is available.

In [6]:
def analyze_response_structure():
    """Analyze and show the complete structure of the JSON response."""
    
    if not ollama_ready:
        print("⚠️  Ollama is not ready, unable to analyze structure")
        return
    
    # Use the Python result if available, otherwise curl
    result = None
    source = ""
    
    if 'python_result' in globals() and python_result is not None:
        result = python_result
        source = "Python"
    elif 'curl_result' in globals() and curl_result is not None:
        result = curl_result
        source = "curl"
    
    if result is None:
        print("❌ No result available for analysis")
        return
    
    print(f"🔍 RESPONSE STRUCTURE ANALYSIS (from {source})")
    print("="*60)
    
    print("📋 Available fields in the response:")
    for key, value in result.items():
        value_type = type(value).__name__
        if isinstance(value, str) and len(value) > 100:
            preview = value[:100] + "..."
        else:
            preview = str(value)
        
        print(f"   - {key} ({value_type}): {preview}")
    
    print("\n📊 Useful information:")
    print(f"   - Model used: {result.get('model', 'N/A')}")
    print(f"   - Creation timestamp: {result.get('created_at', 'N/A')}")
    print(f"   - Total tokens: {result.get('eval_count', 'N/A')}")
    print(f"   - Processing duration: {result.get('eval_duration', 'N/A')} ns")
    print(f"   - Prompt tokens: {result.get('prompt_eval_count', 'N/A')}")
    print(f"   - Generation completed: {result.get('done', 'N/A')}")
    
    # Show the complete formatted response
    print("\n🗂️  Complete JSON response (formatted):")
    print("-"*40)
    print(json.dumps(result, indent=2, ensure_ascii=False))

# Analyze the structure
analyze_response_structure()

🔍 RESPONSE STRUCTURE ANALYSIS (from Python)
📋 Available fields in the response:
   - model (str): gemma3:1b
   - created_at (str): 2025-09-18T08:46:11.219297Z
   - response (str): Okay, let's break down what artificial intelligence (AI) is. It’s a really broad and evolving field,...
   - done (bool): True
   - done_reason (str): stop
   - context (list): [105, 2364, 107, 3689, 563, 16477, 14020, 236881, 106, 107, 105, 4368, 107, 19058, 236764, 1531, 236789, 236751, 2541, 1679, 1144, 16477, 14020, 568, 12553, 236768, 563, 236761, 1030, 236858, 236751, 496, 2126, 6674, 532, 36757, 2135, 236764, 840, 1590, 236858, 236751, 496, 25890, 529, 506, 2307, 14485, 236787, 108, 1018, 236770, 236761, 2640, 1061, 17354, 236787, 27680, 63695, 623, 51836, 236775, 9929, 93205, 1018, 108, 118870, 28243, 563, 51935, 1003, 7107, 12512, 1271, 19443, 236764, 5368, 236764, 653, 37136, 1271, 600, 740, 2121, 9395, 600, 11082, 1660, 3246, 14020, 236761, 1174, 5889, 236789, 236745, 1164, 1003, 3043, 1091, 808, 6

## 🎯 Summary and Next Steps

Congratulations! You have successfully completed your first steps with Ollama. Here’s what you’ve learned:

### ✅ What you did:
1. **Verified** that Ollama is active and running
2. **Made your first request** using Python with the `requests` library
3. **Made the same request** using the `curl` command
4. **Compared** the responses obtained with both methods
5. **Analyzed** the complete structure of the JSON response

### 🔑 Key concepts:
- **API Endpoint**: `http://localhost:11434/api/generate`
- **Basic parameters**: `model`, `prompt`, `stream`
- **JSON response**: contains `response`, token statistics, timing
- **Equivalence**: Python and curl produce the same results

### 🚀 Suggested next steps:
1. **Experiment with different prompts** – try more complex questions
2. **Download other models** – test with `ollama pull llama2` or others
3. **Add parameters** – experiment with `temperature`, `top_p`, `top_k`
4. **Use streaming** – set `stream: true` for real-time responses
5. **Integrate into your applications** – use the Python code as a base

### 📚 Useful resources:
- [Official Ollama Documentation](https://github.com/ollama/ollama)
- [API Reference](https://github.com/ollama/ollama/blob/main/docs/api.md)
- [Available Models](https://ollama.com/library)

### 💡 Tip:
For more complex projects, consider using the notebook `ollama_parameter_guide.ipynb` to explore in detail how different parameters affect response quality!